In [2]:
# import necessary libraries
import util
import numpy as np
import sqlite3
import pandas as pd
import ast

from IPython.display import Image
from IPython.display import display

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVR
from sklearn.ensemble import ExtraTreesClassifier

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

matplotlib.rc("figure", figsize=(8,6))
matplotlib.rc("axes", labelsize=16, titlesize=16)
matplotlib.rc("xtick", labelsize=14)
matplotlib.rc("ytick", labelsize=14)
matplotlib.rc("legend", fontsize=14)
matplotlib.rc("font", size=14)

In [3]:
# load the rating matrix
M = util.load_sparse_csr("imdb_data/rating.npz").toarray()
movie_names = np.load("imdb_data/movies.npy")
user_names = np.load("imdb_data/users.npy")

print M.shape, movie_names.shape, user_names.shape

(18060, 1603) (1603,) (18060,)


In [6]:
new_rating_m, new_user_names, new_movie_names = util.rating_filter(M, user_names, movie_names, (5,5))

print new_rating_m.shape, new_user_names.shape, new_movie_names.shape

(18060,) (1603,)
Original #rating 25658
Remaining #user 358 #movie 504
(358, 504)
Remaining #rating 4629
Sparsity for the rating matrix : 2.57%
(358, 504) (358,) (504,)


In [7]:
train, valid, test, user_mask = util.split_dataset(new_rating_m)
print train.shape, np.sum(train>0), np.sum(np.sum(train, axis=1) == 0)
print valid.shape, np.sum(valid>0), np.sum(np.sum(valid, axis=1) == 0)
print test.shape, np.sum(test>0), np.sum(np.sum(test, axis=1) == 0)

Number of user delete 5
(353, 504) 2669 0
(353, 504) 1040 0
(353, 504) 911 0


In [12]:
DB_NAME = 'imdb_data/imdb_final.db'

In [13]:
# compute validation metric
def compute_mse(prediction, real):
    """ 
    Input:
        prediction (matrix) : prediction of users' ratings
        real (matrix) : real user ratings
    Output:
        mse (double) : mean squared error
    """
    # rule out the empty rating
    return np.mean(((real - prediction)**2)[real.nonzero()])

### 3.x Content-Based Filtering

Content-based filtering methods rely on the description of the item and a profile of the user's preference to make the recommendation. In a content-based recommender system, we firstly use the description of the item to create a vector representation of all the items. Then we need to build a user profile for every users which will indicate what kinds of items this user likes. So the main idea of content-based filtering is that users will like the items that are similar to those them liked in the past (based on their profile). So there are two main problem in order to use the content-based filtering.

* How to generate an abstract representation of items?
    * In our project, items are movies which contains various aspects, including plots, genres, credits, etc.
* How to build a user profile for every users?
    * Generally, a user profile can be built based on user's previous preference and/or user's history interaction with the system. However, in our project, we have no access to the user's interaction with IMDb. So we will only use their history preference.



#### 3.x.1 How to Learn the Content-Based Filtering Model

Let 
* $\theta^{(i)}$ be the profile representation of the i-th user (parameters of the model that we need to learn),
* $m^{(j)}$ be the vector representation of the j-th movie (created from other sources),
* $y^{(i,j)}$ be the rating for the j-th movie given by i-th user,
* $r^{(i,j)}$ be the mask for representing whether i-th user rates j-th motive. $r^{(i,j)} = 1$ means i-th user gives a rating to j-th movie. $r^{(i,j)} = 0$ means otherwiese,
* $N$ be the number of users,
* $M$ be the number of movies,
* $K$ be the dimension of the representation of user profile and movie.

So i-th user's potentional perference about j-th movie can be calculate by in this model:

\begin{align}
{\theta^{(i)}}^T m^{(j)} \notag
\end{align}

In order to learn the best representation of $\theta^{(i)}$ for i-th user, we need to optimize the following loss function based on the training set:

\begin{align}
loss^{(i)} = \frac{1}{2} \sum_{j:r^{(i,j)}=1} ({\theta^{(i)}}^T m^{(j)} - y^{(i,j)})^2 + \frac{\lambda}{2} \sum_{k=1}^{K} (\theta_k^{(i)}) ^2 \notag
\end{align}

In this loss function, we consider the square loss of all rated movies with l2 penalty to avoid overfitting.

Since we have more than one user to consider, the total loss is sum over the loss of all users.

\begin{align}
loss = \frac{1}{2} \sum_{i=1}^{N} \sum_{j:r^{(i,j)}=1} ({\theta^{(i)}}^T m^{(j)} - y^{(i,j)})^2 + \frac{\lambda}{2} \sum_{i=1}^{N} \sum_{k=1}^{K} (\theta_k^{(i)}) ^2 \notag
\end{align} 

So this is our final optimization objective. It's quite similar to the one in linear regression. And in order to minimize such loss we can use gradient descent.

Gradient descent update is

\begin{align}
\theta_k^{(i)} := \theta_k^{(i)} - \alpha (\sum_{j:r^{(i,j)}=1} ({\theta^{(i)}}^T m^{(j)} - y^{(i,j)}) m_k^{(j)} + \lambda \theta_k^{(i)} \notag
\end{align}

In which $\lambda$ is the parameters for control the l2 penalty and $\alpha$ is the learning rate.

In [138]:
class ContentBased:
    def __init__(self, num_user, movie_vec):
        if type(movie_vec) != np.ndarray:
            self.movie_vec = movie_vec.toarray()
        else:
            self.movie_vec = movie_vec
        self.theta = np.zeros((num_user, movie_vec.shape[1]))
        
    def cal_loss(self, rating, lam=0.1):
        pred = self.movie_vec.dot(self.theta.T).T
        mask = rating > 0
        loss = np.sum(np.square(pred - rating) * mask)      
        loss /= 2.0
        loss_l2 = loss + lam / 2.0 * np.sum(np.square(self.theta))
        
        return loss, loss_l2
    
    def cal_grad(self, rating, lam=0.1):
        grad = np.zeros(self.theta.shape)
        mask = rating > 0
        pred = self.movie_vec.dot(self.theta.T).T
        diff = pred - rating
        
        grad = (mask * diff).dot(self.movie_vec)
        
        return grad
    
    def pred(self):
        return self.movie_vec.dot(self.theta.T).T
        
    def train(self, rating, valid=None, max_ite=100, learning_rate=0.2, lam=0.1):
        for i in range(max_ite):
            loss, loss_l2 = self.cal_loss(rating, lam=lam)
            if i%10 == 0:
                if valid is None:
                    print 'Iteration %d train loss: %f' % (i, loss)
                else:
                    v_loss, v_loss_l2 = self.cal_loss(valid, lam=lam)
                    print 'Iteration %d train loss: %f valid loss: %f' % (i, loss, v_loss)
            grad = self.cal_grad(rating, lam)
            self.theta = self.theta - learning_rate * (grad + lam * self.theta)

#### 3.x.2 Use Genres to Represent Movies

So there are many ways to generate the vector representation of the movies. Firstly, let's have a try of genres information. The following code will represent each movie according which genre it belongs to.

In [139]:
def movie_to_vec_genre(dbname, movie_names):
    movie_genres = {}
    all_genres = set()
    
    conn = sqlite3.connect(dbname)
    c = conn.cursor()

    for r in c.execute('''
        SELECT imdb_id, genres
        FROM movie
        '''):
        if r[0] in movie_names:
            gs = ast.literal_eval(r[1])
            if gs is None:
                continue
            all_genres |= set(gs)
            movie_genres[r[0]] = gs
    print len(all_genres), len(movie_genres)
    
    f_genres = list(all_genres)
    movie_vec = np.zeros((len(movie_names), len(f_genres)))
    for i in range(len(movie_names)):
        for g in movie_genres[movie_names[i]]:
            movie_vec[i,f_genres.index(g)] = 1.0
    return movie_vec, f_genres

In [140]:
movie_vec_genres, f_genres = movie_to_vec_genre(DB_NAME, new_movie_names)
print movie_vec_genres.shape
print f_genres

print 'Example genres for movie %s\n%s' % (new_movie_names[0], movie_vec_genres[0,:])

24 504
(504, 24)
[u'Mystery', u'Short', u'Sci-Fi', u'Crime', u'Drama', u'Animation', u'Music', u'Action', u'Comedy', u'Documentary', u'War', u'History', u'Romance', u'Family', u'Horror', u'Thriller', u'Film-Noir', u'Musical', u'Fantasy', u'Adventure', u'News', u'Sport', u'Biography', u'Western']
Example genres for movie tt0456999
[ 0.  0.  0.  1.  0.  0.  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]


As we can see, there are 24 genres in total for all 504 movies. Let's use this metrix as the representation of movies for our Content-Based filtering.

In [141]:
cb = ContentBased(train.shape[0], movie_vec_genres)
cb.train(train, valid=valid, max_ite=50, learning_rate=0.01, lam=0.1)

mse_train = compute_mse(cb.pred(), train)
mse_valid = compute_mse(cb.pred(), valid)

print 'MSE at train %f valid %f' % (mse_train, mse_valid)

Iteration 0 train loss: 63022.000000 valid loss: 26524.500000
Iteration 10 train loss: 16612.620059 valid loss: 12012.547503
Iteration 20 train loss: 9326.644622 valid loss: 9502.152275
Iteration 30 train loss: 6656.762464 valid loss: 8633.983846
Iteration 40 train loss: 5330.488536 valid loss: 8279.680470
MSE at train 3.408181 valid 15.638814


As we can see from the results, we can achieve about 15.639 mean square error when use this model to predict user's perference about movies and give recommendation. It means on average difference between the ratings given by this model and the actual ratings is abount 3.95. (rating are in range of 1 to 10)

#### 3.x.3 Use Plots to Represent Movies

There are only 24 genres, how about tring some richer information? Movie plots might be a good choice since usually it contains large chunk of text. We choice to use the tf-idf algorithm to try plots into vector representation.

In [126]:
def movie_to_vec_plot(dbname, movie_names):
    plots = util.load_movie_plot(dbname, movie_names)
    all_plots = []
    for i in range(len(movie_names)):
        p = ''
        for plot in plots[movie_names[i]]:
            p += plot + ' '
        all_plots.append(p)
    # tokenizing
    count_vect = CountVectorizer()
    plot_counts = count_vect.fit_transform(all_plots)
    # tfidf
    tf_transformer = TfidfTransformer().fit(plot_counts)
    plot_tfidf = tf_transformer.transform(plot_counts)    
    
    return all_plots, plot_counts, plot_tfidf, count_vect.get_feature_names()

In [127]:
all_plots, plot_counts, plot_tfidf, vocabulary = movie_to_vec_plot(DB_NAME, new_movie_names)
print len(all_plots), plot_counts.shape, plot_tfidf.shape, len(vocabulary)

print 'Example plots for movie %s\n%s' % (new_movie_names[0], all_plots[0])

504 (504, 9499) (504, 9499) 9499
Example plots for movie tt0456999
The film deals with a boisterous undercover female cop who gets sent to a high school in order to get close to a criminal in hideout by befriending his teenage daughter. The general set up and the fighting antics of the female cop play close resemblance to Stephen Chow's classic Fight Back to School. 


As we see, the vocabulary size of the plots we have it quite large (9499 distinct words). If we directly use such representation.

In [128]:
cb = ContentBased(train.shape[0], plot_tfidf)
cb.train(train, valid=valid, max_ite=50, lam=0.1)

mse_train = compute_mse(cb.pred(), train)
mse_valid = compute_mse(cb.pred(), valid)

print 'MSE at train %f valid %f' % (mse_train, mse_valid)

Iteration 0 train loss: 63022.000000 valid loss: 26524.500000
Iteration 10 train loss: 4401.264583 valid loss: 11945.853378
Iteration 20 train loss: 4114.034755 valid loss: 11798.938670
Iteration 30 train loss: 4096.210943 valid loss: 11794.504433
Iteration 40 train loss: 4094.333932 valid loss: 11794.474821
MSE at train 3.067886 valid 22.681732


As we can see, using all 9499 features will easily lead to overfit. We can achieve a very good performance on the training set. However on the validation set, our performance is quite bad. It's because since the dimension of user profile and movie representation are the same. We have $N*K = 358 * 9499 = 3.4M$ parameters to learn but we only have about 2 thousand training examples. So it's necessary to do feature selection in this case to speed up the training process and avoid overfitting.

In the following we choose to use coefficients of a linear model to select the features. We fit a linear support vector regression model to predict the average score of the movies with the tf-idf feature from movie plots. And then only keep features with high coefficients.

In [129]:
def feature_selection(X, y, th='7*mean', C=0.03):
    lsvr = LinearSVR(C=C).fit(X, y)
    model = SelectFromModel(lsvr, prefit=True, threshold='7*mean')
    X_new = model.transform(X)
    print X_new.shape

    return X_new, lsvr.coef_

In [134]:
y = train.sum(axis=0) / ((train>0).sum(axis=0)+1e-4)
movie_vec_plot_sel, coef = feature_selection(plot_tfidf, y)

# print 'Top 10 coef terms'
# print len(vocabulary)
# top_idx = coef.argsort()[-71:][::-1]
# for idx in top_idx:
#     print vocabulary[idx], 

(504, 71)


As we can see, finally we only keep 71 features out of 9499 features. If we run the Content-Based Filtering again, we have

In [131]:
cb = ContentBased(train.shape[0], movie_vec_plot_sel)
cb.train(train, valid=valid, max_ite=50, lam=0.1)

mse_train = compute_mse(cb.pred(), train)
mse_valid = compute_mse(cb.pred(), valid)

print 'MSE at train %f valid %f' % (mse_train, mse_valid)

Iteration 0 train loss: 63022.000000 valid loss: 26524.500000
Iteration 10 train loss: 13345.818953 valid loss: 8593.730959
Iteration 20 train loss: 10236.443355 valid loss: 7225.748653
Iteration 30 train loss: 9213.591353 valid loss: 6852.688442
Iteration 40 train loss: 8710.132396 valid loss: 6716.331497
MSE at train 6.306739 valid 12.803903


The performance is greatly improved even with much fewer features.

In [143]:
from fastFM.mcmc import FMClassification, FMRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.datasets import dump_svmlight_file

In [144]:
def fitpredict_logistic(trainX, trainY, testX, classification=True, **params):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    if classification:
        clf = LogisticRegression(**params)
        clf.fit(trainX, trainY)
        return clf.predict_proba(testX)[:, 1]
    else:
        clf = Ridge(**params)
        clf.fit(trainX, trainY)
        return clf.predict(testX)

In [156]:
def fitpredict_libfm(trainX, trainY, testX, classification=True, rank=8, n_iter=100):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    train_file = 'libfm_train.txt'
    test_file = 'libfm_test.txt'
    with open(train_file, 'w') as f:
        dump_svmlight_file(trainX, trainY, f=f)
    with open(test_file, 'w') as f:
        dump_svmlight_file(testX, numpy.zeros(testX.shape[0]), f=f)
    task = 'c' if classification else 'r'
    console_output = !$LIBFM_PATH -task $task -method mcmc -train $train_file -test $test_file -iter $n_iter -dim '1,1,$rank' -out output.libfm
    
    libfm_pred = pandas.read_csv('output.libfm', header=None).values.flatten()
    return libfm_pred

In [157]:
def fitpredict_fastfm(trainX, trainY, testX, classification=True, rank=8, n_iter=100):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    if classification:
        clf = FMClassification(rank=rank, n_iter=n_iter)
        return clf.fit_predict_proba(trainX, trainY, testX)
    else:
        clf = FMRegression(rank=rank, n_iter=n_iter)
        return clf.fit_predict(trainX, trainY, testX) 

In [158]:
def convert(rating):
    mask = rating > 0
    
    nz = rating.nonzero()
    
    X = np.concatenate((nz[0].reshape(-1,1), nz[1].reshape(-1,1)), axis=1)
    y = rating[nz]
    
    print X.shape, y.shape
    return X, y
    
X_train, y_train = convert(train)
X_valid, y_valid = convert(valid)

y_pred = fitpredict_logistic(X_train, y_train, X_valid, False)

print np.mean((y_valid - y_pred) ** 2) 

y_pred = fitpredict_fastfm(X_train, y_train, X_valid, False)

print np.mean((y_valid - y_pred) ** 2) 

(2669, 2) (2669,)
(1040, 2) (1040,)
3.34519592153
3.42285020189


## 4. Reference
* https://www.coursera.org/learn/machine-learning/lecture/uG59z/content-based-recommendations